
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [ ]:
import pandas as pd
import numpy as np

#Load
file_path = 'data/Corona_NLP_train.csv'  # Adjust the path if necessary
data = pd.read_csv(file_path, encoding='latin1')

filtered_data = data[data['Sentiment'] != 'Neutral']
X = filtered_data['OriginalTweet'].values
Y = np.where(filtered_data['Sentiment'].str.contains('Positive'), 1, 0)

n_samples = len(X)
train_end = int(n_samples * 0.6)
test_end = int(n_samples * 0.75)

X_train = X[:train_end]
Y_train = Y[:train_end]
X_test = X[train_end:test_end]
Y_test = Y[train_end:test_end]
X_valid = X[test_end:]
Y_valid = Y[test_end:]

print(f"Training set: {len(X_train)} samples")
print(f"Testing set: {len(X_test)} samples")
print(f"Validation set: {len(X_valid)} samples")


Training set: 20066 samples
Testing set: 5017 samples
Validation set: 8361 samples


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

model = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', LogisticRegression(max_iter=200, solver='liblinear'))
])

model.fit(X_train, Y_train)
sample_probs = model.predict_proba(X_test[:5])
print("Sample Predicted Probabilities:", sample_probs)


Sample Predicted Probabilities: [[0.87768301 0.12231699]
 [0.99712858 0.00287142]
 [0.03465924 0.96534076]
 [0.79880644 0.20119356]
 [0.43905477 0.56094523]]


In [ ]:
from sklearn.metrics import precision_score, recall_score

Y_pred = model.predict(X_test)
precision_0 = precision_score(Y_test, Y_pred, pos_label=0)
precision_1 = precision_score(Y_test, Y_pred, pos_label=1)
recall_0 = recall_score(Y_test, Y_pred, pos_label=0)
recall_1 = recall_score(Y_test, Y_pred, pos_label=1)

print(f"Precision (Class 0): {precision_0}")
print(f"Precision (Class 1): {precision_1}")
print(f"Recall (Class 0): {recall_0}")
print(f"Recall (Class 1): {recall_1}")


Precision (Class 0): 0.8579595426561126
Precision (Class 1): 0.8709442216551221
Recall (Class 0): 0.8464208242950109
Recall (Class 1): 0.8808997050147492


In [ ]:
def cost(model, threshold, X, Y):
    probabilities = model.predict_proba(X)[:, 1]
    predictions = (probabilities >= threshold).astype(int)    
    costs = np.where(
        (predictions == 1) & (Y == 0), 
        5,
        np.where(
            (predictions == 0) & (Y == 1),
            1,
            0 
        )
    )
    
    return costs.mean()
example_cost = cost(model, threshold=0.5, X=X_test, Y=Y_test)
print(f"Average Cost (Threshold=0.5): {example_cost}")


Average Cost (Threshold=0.5): 0.4171815826190951


In [ ]:
thresholds = np.linspace(0, 1, 101)
costs = [cost(model, threshold, X_test, Y_test) for threshold in thresholds]
optimal_threshold_index = np.argmin(costs)
optimal_threshold = thresholds[optimal_threshold_index]
cost_at_optimal_threshold = costs[optimal_threshold_index]

print(f"Optimal Threshold: {optimal_threshold}")
print(f"Cost at Optimal Threshold: {cost_at_optimal_threshold}")


Optimal Threshold: 0.8
Cost at Optimal Threshold: 0.27267291209886385


In [ ]:
import math

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)
n_valid = len(Y_valid)
epsilon = math.sqrt((1 / (2 * n_valid)) * math.log(2 / 0.01))
cost_interval_valid = (
    cost_at_optimal_threshold_valid - epsilon,
    cost_at_optimal_threshold_valid + epsilon
)
assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

print(f"Cost at Optimal Threshold (Validation): {cost_at_optimal_threshold_valid}")
print(f"99% Confidence Interval: {cost_interval_valid}")


Cost at Optimal Threshold (Validation): 0.27747877048199976
99% Confidence Interval: (0.25967857077556256, 0.29527897018843696)


In [ ]:
probabilities_valid = model.predict_proba(X_valid)[:, 1]
predictions_valid = (probabilities_valid >= optimal_threshold).astype(int)
C = (1 - (predictions_valid)) * Y_valid + 5 * (1 - Y_valid) * predictions_valid

variance_of_C = np.var(C, ddof=1) 
mean_C = np.mean(C)
b = 5 
n_valid = len(C)
epsilon_bennett = math.sqrt((2 * variance_of_C * math.log(2 / 0.01)) / n_valid) + (b * math.log(2 / 0.01)) / (3 * n_valid)
interval_of_C = (
    mean_C - epsilon_bennett,
    mean_C + epsilon_bennett
)
assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)

print(f"Empirical Variance of C: {variance_of_C}")
print(f"Confidence Interval using Bennett's inequality: {interval_of_C}")


Empirical Variance of C: 0.669407805320785
Confidence Interval using Bennett's inequality: (0.24729531160092996, 0.30766222936306953)
